# Finding Similar and Duplicate Images

In [7]:
from PIL import Image
from PIL import ImageStat
import itertools
import numpy as np
import os

In [3]:
os.chdir(r'C:\Users\Lenovo\Desktop\python\task_rails\dev_dataset')
os.getcwd()  

'C:\\Users\\Lenovo\\Desktop\\python\\task_rails\\dev_dataset'

In [4]:
# Список изображений, с которыми будем работать
files_list = os.listdir(r'C:\Users\Lenovo\Desktop\python\task_rails\dev_dataset')

# 1. Найдем дубликаты изображений. 

Для этого будем использовать попиксельное эвклидово расстояние между двумя изображениями. 
Это расстояние должно быть равно нулю.

Введем вспомогательные функции:

In [5]:
def image_filter(image):
    image = Image.open(image)
    image = image.convert('L')# Убираем цвет. Переводим изображение в градации серого. 
    image = image.resize((8,8), Image.LANCZOS) # Уменьшаем картинку до 8х8, тогда общее число пикселей составит 64 .
    return image

In [6]:
def dist(im_A, im_B):
    im_A = np.array(im_A)
    im_B = np.array(im_B)
    err = np.sum((im_A - im_B) ** 2)
    return err

In [8]:
def duplicates(image_list):
    same = []
    for filename_A, filename_B in itertools.combinations(image_list, 2):
        im_A = image_filter(filename_A)
        im_B = image_filter(filename_B)
        if dist(im_A, im_B) == 0:
            same.append((filename_A, filename_B))
    return same 

Найдем дубликаты изображений в нашем списке.

In [9]:
d = duplicates(files_list)
d

[('1.jpg', '1_duplicate.jpg'), ('11.jpg', '11_duplicate.jpg')]


# 2. Найдем подобные изображения.

Для этого будем использовать хеш-функцию Average Hash каждого изображения. Далее схожесть картинок будем определять по расстоянию Хэммига (Hamming Distance) между ними. Чем ближе значение расстояния Хемминга к нулю, тем более идентичные изображения. (Его можно было использовать и при поиске дубликатов).

Введем вспомогательные функции:

In [10]:
def averagehash(image):
    averageValue = ImageStat.Stat(image).mean[0] # Находим среднее среднее значение яркости получившегося изображения.
    averageHash = 0
    for row in xrange(8):
        for col in xrange(8):
            averageHash <<= 1
            averageHash |= 1 * ( image.getpixel((col, row)) >= averageValue)
#Бинаризация картинки. Оставляем только те пиксели, которые больше среднего (считаем их за 1, а все остальные за 0).
#Строим хэш. Переводим полученные 64 значений 1 и 0 картинки в одно значение хэша. 
    return str(averageHash)

In [11]:
# Расстояние Хэмминга — число позиций, в которых соответствующие цифры двух строк различны.
def hamming(s_1, s_2):
    return float(sum(c1 != c2 for c1, c2 in zip(s_1, s_2))) / float(len(s_1))


In [12]:
def modification(image_list):
    modificate = []
    for filename_A, filename_B in itertools.combinations(image_list, 2):
        im_A = image_filter(filename_A)
        im_B = image_filter(filename_B)
        im_A_hash = averagehash(im_A)
        im_B_hash = averagehash(im_B)
        if hamming(im_A_hash, im_B_hash) < 0.1 and (filename_A, filename_B) not in duplicates(image_list):
            modificate.append((filename_A, filename_B))
    return modificate

Найдем схожие картинки, исключим из их списка найденые дубликаты.

In [13]:
m = modification(files_list)
m

[('11.jpg', '11_modification.jpg'),
 ('11_duplicate.jpg', '11_modification.jpg'),
 ('15.jpg', '15_modification.jpg')]

Очевидно, изображения типа similar более различны попиксельно(чем выше рассмотреные duplicate и modificate) так как на них есть сдвиг и\или другой угол обзора, что больше нарушает пиксельное соответствие. Попробуем понизить требуемый уровень расстояния Хэмминга до 0.2.

In [14]:
def similar(image_list):
    sim = []
    for filename_A, filename_B in itertools.combinations(image_list, 2):
        im_A = image_filter(filename_A)
        im_B = image_filter(filename_B)
        im_A_hash = averagehash(im_A)
        im_B_hash = averagehash(im_B)
        if hamming(im_A_hash, im_B_hash) < 0.2 and (filename_A, filename_B) not in d and (filename_A, filename_B) not in m :
            sim.append((filename_A, filename_B))
    return sim

In [15]:
s = similar(files_list)
s

[('6.jpg', '6_similar.jpg')]

# Спасибо, задание было интересным и познавательным!